In [10]:
import os
import nltk

In [12]:
corpus_path=os.path.abspath("../data/NKJP_1.2_nltk/")
corpus=nltk.corpus.reader.TaggedCorpusReader(root=corpus_path, fileids=".*")

In [13]:
tags = [x[1] for x in corpus.tagged_words()]
fqd = nltk.FreqDist(tags)



In [14]:
# @TODO merge tags so that their cardinalities are ≥  100, 1000
sets = fqd.most_common()
sets = [(s[0].split(':'), s[1]) for s in sets]
sets = [(s[0][0], s[0][1:], s[1]) for s in sets]
# sets = {s[0][0]:{'tags':s[0][1:], 'card': s[1]} for s in sets}
# we want a struct:
# main_tag : {
# list(
#     {tags: [other tags], card: cardinality}
# )
# }
structurized_data = {}
for s in sets:
    k = s[0]
    v = {'tags': s[1], 'card': s[2]}
    if k in structurized_data.keys():
        structurized_data[k]+=[v]
    else:
        structurized_data[k] =[v]

We see that:
QUB has 1 optional tag, we'll want to remove it since they make a really small difference.  
PREP has 1 optional tag, but the sizes are ok, so we'll ignore it  
ADV is good as it is  
PRAET has 1 optional tag  
PPRON12 has 1 optional tag  

why am I mentioning this?
because the missing tags were breaking my naive algorithm lol.

In [15]:
# remove optional tags - in most cases their respective set sizes are too small anyway
keys_with_optional_tags = {'PREP', 'PRAET', 'PPRON12'}

def get_num_of_mandatory_tags(A):
    min_len = 0
    for el in A:
        if(len(el['tags'])<min_len):
            min_len = len(el['tags'])
    
    return min_len

for k in keys_with_optional_tags:
    flexem = structurized_data[k]
    mandatory_tags = get_num_of_mandatory_tags(flexem)
    for el in flexem:
        if len(el['tags'])>mandatory_tags:
            el['tags'] = el['tags'][:-1]

In [25]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 


def concat(A):
    res = ""
    for a in A:
        res += ':'
        res += a
    
    res = res[1:]
    return res


def flatten(A):
    def merge(B):
        tmp = {'tags': B[0]['tags'], 'card': 0}
        for el in B:
            tmp['card'] += el['card']    
        return tmp
    
    sets = {}
    for el in A:
        key = concat(el['tags'])
        if key in sets:
            sets[key] += [el]
        else:
            sets[key] = [el]
    
    res = []
    for k, v in sets.items():
        res += [merge(v)]
    
    return res


from queue import *
result = {}
for k in structurized_data:
    t = structurized_data[k].copy()
    q = PriorityQueue()

    t = sorted(t, key=lambda x: x['card'])

    
    p = t[0]
    fin = []
    while(p['card']<100 and len(t)>0):
        p = t[0]
        best = (0,0, None, None)
        for i in range(1, len(t)):
            r = t[i]
            pr_tags = intersection(p['tags'], r['tags'])
            pr_card = p['card'] + r['card']
            intersection_size = len(pr_tags)
            if(intersection_size > best[0]):
                best = (intersection_size, pr_card, {'tags': pr_tags, 'card': pr_card}, i)
            elif (intersection_size == best[0] and pr_card <= best[1]):
                best = (intersection_size, pr_card, {'tags': pr_tags, 'card': pr_card}, i)
                
        if best[3] is not None:
            del(t[best[3]])
            del(t[0])
    
            t.append(best[2])
            t = sorted(t, key=lambda x: x['card'])
    
        else:
            fin += [t[0]]
            del(t[0])
    

    
    t += fin
    print('---------------------')
    print(k)
    if k in result:
        result[k] += t
    else:
        result[k] = t
    for _ in t:
        print(_)
    

---------------------
INTERP
{'tags': [], 'card': 223499}
---------------------
QUB
{'tags': ['WOK'], 'card': 31}
{'tags': ['NWOK'], 'card': 35}
{'tags': [], 'card': 68402}
---------------------
CONJ
{'tags': [], 'card': 44953}
---------------------
FIN
{'tags': ['PL', 'PRI', 'PERF'], 'card': 573}
{'tags': ['SEC'], 'card': 740}
{'tags': ['SG', 'PRI', 'PERF'], 'card': 939}
{'tags': ['PL', 'TER', 'PERF'], 'card': 1219}
{'tags': ['SG', 'SEC', 'IMPERF'], 'card': 2426}
{'tags': ['PL', 'PRI', 'IMPERF'], 'card': 2598}
{'tags': ['SG', 'TER', 'PERF'], 'card': 3860}
{'tags': ['SG', 'PRI', 'IMPERF'], 'card': 7302}
{'tags': ['PL', 'TER', 'IMPERF'], 'card': 9599}
{'tags': ['SG', 'TER', 'IMPERF'], 'card': 30242}
---------------------
SUBST
{'tags': ['PL', 'LOC', 'M1'], 'card': 155}
{'tags': ['PL', 'DAT', 'M3'], 'card': 185}
{'tags': ['PL', 'DAT', 'N'], 'card': 191}
{'tags': ['M2'], 'card': 201}
{'tags': ['PL', 'ACC', 'M2'], 'card': 298}
{'tags': ['SG', 'VOC'], 'card': 306}
{'tags': ['PL', 'DAT', 'F'

In [26]:
#save result
import json

with open('pos.json', 'w') as file:
     file.write(json.dumps(result)) # use `json.loads` to do the reverse
        
result

{'INTERP': [{'tags': [], 'card': 223499}],
 'QUB': [{'tags': ['WOK'], 'card': 31},
  {'tags': ['NWOK'], 'card': 35},
  {'tags': [], 'card': 68402}],
 'CONJ': [{'tags': [], 'card': 44953}],
 'FIN': [{'tags': ['PL', 'PRI', 'PERF'], 'card': 573},
  {'tags': ['SEC'], 'card': 740},
  {'tags': ['SG', 'PRI', 'PERF'], 'card': 939},
  {'tags': ['PL', 'TER', 'PERF'], 'card': 1219},
  {'tags': ['SG', 'SEC', 'IMPERF'], 'card': 2426},
  {'tags': ['PL', 'PRI', 'IMPERF'], 'card': 2598},
  {'tags': ['SG', 'TER', 'PERF'], 'card': 3860},
  {'tags': ['SG', 'PRI', 'IMPERF'], 'card': 7302},
  {'tags': ['PL', 'TER', 'IMPERF'], 'card': 9599},
  {'tags': ['SG', 'TER', 'IMPERF'], 'card': 30242}],
 'SUBST': [{'tags': ['PL', 'LOC', 'M1'], 'card': 155},
  {'tags': ['PL', 'DAT', 'M3'], 'card': 185},
  {'tags': ['PL', 'DAT', 'N'], 'card': 191},
  {'tags': ['M2'], 'card': 201},
  {'tags': ['PL', 'ACC', 'M2'], 'card': 298},
  {'tags': ['SG', 'VOC'], 'card': 306},
  {'tags': ['PL', 'DAT', 'F'], 'card': 346},
  {'tags'